<a href="https://colab.research.google.com/github/aa-amory/AIESI/blob/master/AIESI_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/aa-amory/AIESI.git

fatal: destination path 'AIESI' already exists and is not an empty directory.


# PIP all required lib

In [2]:
!pip install -U git+https://github.com/madmaze/pytesseract.git
!sudo apt install tesseract-ocr
!pip install colorama
!pip install opencv_wrapper

  Cloning https://github.com/madmaze/pytesseract.git to /tmp/pip-req-build-937lufze
  Running command git clone -q https://github.com/madmaze/pytesseract.git /tmp/pip-req-build-937lufze
  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=14106 sha256=3cb7399127064ece952ce14febf2840fc470a8b30514f0b41516cba23fef660e
  Stored in directory: /tmp/pip-ephem-wheel-cache-snh3aeok/wheels/be/2a/a1/a40dbc7e579dffb2be8bbc3243c491de2d132899309f008b1f
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bi

# Import libs

In [1]:
import json
import os
import random
from os import path
from string import ascii_uppercase, digits, punctuation
import colorama
import regex
import torch
from torch import nn
import pytesseract
import cv2
import argparse
import numpy as np
import opencv_wrapper as cvw
from skimage.filters import threshold_local

### Hyper- parameters

In [2]:
device= 'cpu'
hidden_size = 256
VOCAB= ascii_uppercase+digits+punctuation+" \t\n"
device= torch.device('cpu')

# full code

In [3]:
def get_acc(directory, path):
    font     = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5
    fontColor  = (255,0,0)
    lineType = 1
    path = directory+path
    # path = args['image']
    # op_path = args['output']

    op_path = directory
    if op_path[-1]!='/':
    	op_path.append('/')


    #Threshold
    image = cv2.imread(path)

    height,width,channel = image.shape

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    T = threshold_local(gray, 15, offset = 6, method = "gaussian") # generic, mean, median, gaussian
    thresh = (gray > T).astype("uint8") * 255
    thresh = ~thresh

    #cv2.imwrite(op_path+'threshold.png', thresh)

    #Dilation
    kernel =np.ones((1,1), np.uint8)
    ero = cv2.erode(thresh, kernel, iterations= 1)
    img_dilation = cv2.dilate(ero, kernel, iterations=1)
    #cv2.imwrite(op_path+'dilated.png', img_dilation)

    # Remove noise
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(img_dilation, None, None, None, 8, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    final = np.zeros((labels.shape), np.uint8)
    for i in range(0, nlabels - 1):
        if sizes[i] >= 10:   #filter small dotted regions
            final[labels == i + 1] = 255
    #cv2.imwrite(op_path+'final.png', final)
    #Find contours
    kern = np.ones((5,15), np.uint8)
    img_dilation = cv2.dilate(final, kern, iterations = 1)
    #cv2.imwrite(op_path+'contours.png', img_dilation)
    contours, hierarchy = cv2.findContours(img_dilation, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # Map contours to bounding rectangles, using bounding_rect property
    rects = map(lambda c: cv2.boundingRect(c), contours)
    # Sort rects by top-left x (rect.x == rect.tl.x)
    sorted_rects = sorted(rects, key =lambda r: r[0])
    sorted_rects = sorted(sorted_rects, key =lambda r: r[1])

    tt = image.copy()
    dictionary = {}
    etfo=''
    for i,rect in enumerate(sorted_rects):
        temp_dic = {}
        x,y,w,h = rect
        if(w<20 or h<20):
            continue
        temp_dic['coords'] = [x,y,w,h]
        words = []
        temp = tt[y:y+h, x:x+w]
        #cv2.imwrite('/content/gdrive/My Drive/ResearchPaper/AIESI_complete_pipeline/temp/'+str(i)+'.png',temp)
        temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
        hi = pytesseract.image_to_data(temp, config=r'--psm 6')
        hi = hi.split()
        ind = 22
        while(True):
            if (ind>len(hi)):
                break
            if(int(hi[ind])==-1):
                ind+=11
            else:
                #cv2.putText(image,hi[ind]+','+hi[ind+1], (x,y), font, fontScale,fontColor,lineType)
                tem = {}
                tem['confidence'] = hi[ind]
                tem['text'] = hi[ind+1]
                etfo=etfo+hi[ind+1]
                etfo=etfo+" "
                x+=len(hi[ind+1])*20
                ind+=12
                words.append(tem)
        temp_dic['words'] = words
        etfo=etfo+'\n'
        #cvw.rectangle(image, rect, cvw.Color.GREEN, thickness=1)
        dictionary[i] = temp_dic


    cv2.imwrite(op_path+"result.png", image)
    return json.dumps(dictionary),etfo


class MyModel0(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=2, bidirectional=True)
        self.linear = nn.Linear(hidden_size * 2, 5)

    def forward(self, inpt):
        embedded = self.embed(inpt)
        feature, _ = self.lstm(embedded)
        oupt = self.linear(feature)
        return oupt

def get_test_data(etfo_input):
    
    text = etfo_input
    text_tensor = torch.zeros(len(text), 1, dtype=torch.long)
    text_tensor[:, 0] = torch.LongTensor([VOCAB.find(c) for c in text])

    return text_tensor.to(device)

def pred_to_dict(text, pred, prob):
    res = {"company": ("", 0), "date": ("", 0), "address": ("", 0), "total": ("", 0)}
    keys = list(res.keys())

    seps = [0] + (np.nonzero(np.diff(pred))[0] + 1).tolist() + [len(pred)]
    for i in range(len(seps) - 1):
        pred_class = pred[seps[i]] - 1
        if pred_class == -1:
            continue

        new_key = keys[pred_class]
        new_prob = prob[seps[i] : seps[i + 1]].max()
        if new_prob > res[new_key][1]:
            res[new_key] = (text[seps[i] : seps[i + 1]], new_prob)

    return {k: regex.sub(r"[\t\n]", " ", v[0].strip()) for k, v in res.items()}


def test(text_tensor):
    print('-------------------------------loged test----------------------------------')
    model = MyModel0(len(VOCAB), 16, hidden_size).to(device)
    # dataset = MyDataset(None, args.device, test_path="/content/gdrive/My Drive/ResearchPaper/KIPE/data/test_dict.pth")

    model.load_state_dict(torch.load("/content/AIESI/AIESI_using_tesseract/model.pth"))

    model.eval()
  
    with torch.no_grad():
            oupt = model(text_tensor)
            prob = torch.nn.functional.softmax(oupt, dim=2)
            prob, pred = torch.max(prob, dim=2)
            prob = prob.squeeze().cpu().numpy()
            pred = pred.squeeze().cpu().numpy()
            real_text = etfo
            #real_text=text_tensor
            result = pred_to_dict(real_text, pred, prob)

            with open("/content/AIESI/AIESI_using_tesseract/results" + 'result.txt' + ".txt", "w", encoding="utf-8") as json_opened:
                json.dump(result, json_opened, indent=4)

            print('-------------------------------result----------------------------------')
            print(result)
            #print(key)


def pre_text(img_name):
  gson_data, etfo = get_acc('/content/AIESI/invoices/',img_name)
  print('----------------gson_data---------------------------')
  print(gson_data)


  
  temp_text=''
  for i in range(len(etfo)):
      temp_text=temp_text+etfo[i]
  etfo=temp_text
  etfo= etfo.upper()
  print('--------------------etfo-----------------------')
  print(etfo)
  #text_tensor = get_test_data()
  text_tensor = get_test_data(etfo)
  print('-------------------text_tensor------------------------')
  print(text_tensor.shape)
  '''for i in range(0,(len(text_tensor)-1)):
    if text_tensor[i]<0 or text_tensor[i]>70:
      text_tensor = torch.cat([text_tensor[0:i], text_tensor[i+1:]])
    ''' 
  text_tensor=text_tensor[text_tensor[:, 0] > 0]
  text_tensor=text_tensor[text_tensor[:, 0] < 70]
  return text_tensor,etfo

In [ ]:
#KVE for single image
img_name='1003-receipt.jpg'
text_tsr,etfo=pre_text(img_name)
test(text_tsr)

In [5]:
#KVE for batch of images
from os import walk
filenames='start_itr'
_, _, filenames = next(walk('/content/AIESI/invoices/'),"itr_end")
for f in range(0,len(filenames)-1):
  print('----------'+filenames[f]+'--------------')
  if filenames[f]!='inv_list.txt':
    img_name=filenames[f]
    text_tsr,etfo=pre_text(img_name)
    test(text_tsr)

Streaming output truncated to the last 5000 lines.
2,50 
COKE 
DIET 
J: 
10.00 
14 HANDS CAB 
7 
G 
62.50 
SUBTOTAL 
, 
5.39 
TAX 
.8B9 
CR. 
TOTAL 
CREDIT/DEBIT 
MINIMUM 
$20 
PAYMENT 
THANK 
FOR 
JOINING 
TODAY. 
YOU 
US 

-------------------text_tensor------------------------
torch.Size([508, 1])
-------------------------------loged test----------------------------------
-------------------------------result----------------------------------
{'company': '\\  A=  I,  , 2', 'date': 'A A ——', 'address': '', 'total': ''}
----------1144-receipt.jpg--------------
----------------gson_data---------------------------
{"0": {"coords": [130, 0, 620, 1000], "words": [{"confidence": "54", "text": "CHLOEBI"}, {"confidence": "26", "text": "oe"}, {"confidence": "95", "text": "81"}, {"confidence": "95", "text": "Ludlow"}, {"confidence": "85", "text": "st"}, {"confidence": "2", "text": "cP"}, {"confidence": "11", "text": "regen"}, {"confidence": "96", "text": "New"}, {"confidence": "96", "text": "Yo